In [1]:
import os
from dotenv import load_dotenv
load_dotenv() #loading all the environment variables

groq_api_key = os.getenv("GROQ_API_KEY")
groq_api_key    

'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'

In [2]:
!pip install langchain_groq

In [5]:
from langchain_groq import ChatGroq

model = ChatGroq(model="llama-3.3-70b-versatile",groq_api_key=groq_api_key)
model

ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 32768, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': False, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x103dafc70>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x1283851e0>, model_name='llama-3.3-70b-versatile', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [6]:
from langchain_core.messages import HumanMessage
from langchain_core.messages import AIMessage

response = model.invoke(
    [HumanMessage(content="Hi, My name is Ratnesh Bhardwaj and I am an AI Engineer.")]
)

response.content

"Nice to meet you, Ratnesh Bhardwaj. It's great to hear that you're an AI Engineer. That's a fascinating field, and I'm sure you're doing some cutting-edge work. What specific areas of AI are you interested in or currently working on? Are you more into machine learning, natural language processing, computer vision, or something else? I'm here to chat and learn more about your work."

In [7]:
model.invoke(
    [HumanMessage(content="Hi, My name is Ratnesh Bhardwaj and I am an AI Engineer."),
     AIMessage(content="Nice to meet you, Ratnesh Bhardwaj. It's great to hear that you're an AI Engineer."),
     HumanMessage(content="Hey! What's my name? and what do i do?")]
)

AIMessage(content="Your name is Ratnesh Bhardwaj, and you're an AI Engineer.", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 102, 'total_tokens': 121, 'completion_time': 0.032763593, 'completion_tokens_details': None, 'prompt_time': 0.008115889, 'prompt_tokens_details': None, 'queue_time': 0.058464035, 'total_time': 0.040879482}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_68f543a7cc', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019bb847-f5c6-7730-89dc-27394d64d2d0-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 102, 'output_tokens': 19, 'total_tokens': 121})

In [8]:
!pip install langchain_community

In [12]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables import RunnableWithMessageHistory

store = {}

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

In [13]:
with_message_history = RunnableWithMessageHistory(model, get_session_history=get_session_history)

config = {
    'configurable': {
        "session_id":"chat1"
    }
}

In [15]:
response = with_message_history.invoke(
    [HumanMessage(content="Hi, My name is Ratnesh Bhardwaj and I am an AI Engineer.")], 
     config=config
)

In [16]:
response.content 

'Hello again, Ratnesh Bhardwaj. Welcome back. As an AI Engineer, you must be quite familiar with the latest advancements in artificial intelligence and machine learning. What kind of projects have you been working on lately? Are you more focused on research and development or implementing AI solutions for real-world applications?'

In [17]:
response = with_message_history.invoke(
    [HumanMessage(content="Hi, What's My Name and what do I do?.")], 
     config=config
)

In [18]:
response.content

"Your name is Ratnesh Bhardwaj, and you're an AI Engineer."

In [19]:
 store

{'chat1': InMemoryChatMessageHistory(messages=[HumanMessage(content='Hi, My name is Ratnesh Bhardwaj and I am an AI Engineer.', additional_kwargs={}, response_metadata={}), AIMessage(content="Nice to meet you, Ratnesh Bhardwaj. It's great to hear that you're an AI Engineer. That's a fascinating field, and I'm sure you must be working on some exciting projects. What specific areas of AI do you specialize in, such as machine learning, natural language processing, or computer vision?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 68, 'prompt_tokens': 54, 'total_tokens': 122, 'completion_time': 0.176631828, 'completion_tokens_details': None, 'prompt_time': 0.005451869, 'prompt_tokens_details': None, 'queue_time': 0.16196807, 'total_time': 0.182083697}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_45180df409', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019bb851-e0ed-7b1

In [29]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

In [31]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system",
         "You are a helpful assitant and answer all the questions as per your best knoeledge.",
         ),
     MessagesPlaceholder(variable_name="messsages")
    ]
)

chain = prompt|model



In [33]:
response = chain.invoke(
    {"messsages":[HumanMessage(content="Hi, My name is Ratnesh Bhardwaj!")]}
)

response.content

"Nice to meet you, Ratnesh Bhardwaj! It's a pleasure to interact with you. Is there something I can help you with or would you like to chat? I'm all ears!"

In [36]:
with_message_history = RunnableWithMessageHistory(chain, get_session_history)

config1 = {
    "configurable":{
        "session_id":"chat2"
    }
}

response = with_message_history.invoke(
    [HumanMessage(content="Hi, My name is Ratnesh Bhardwaj!")],
    config=config1,
)

response.content

"Hello Ratnesh Bhardwaj! It's nice to meet you. Is there something I can help you with or would you like to chat? I'm all ears!"

In [37]:
##Adding more complexity
 
prompt = ChatPromptTemplate.from_messages(
    [
        ("system",
         "You are a helpful assitant and answer all the questions as per your best knoeledge in {language}",
         ),
     MessagesPlaceholder(variable_name="messsages")
    ]
)

chain = prompt | model

In [39]:
response = chain.invoke(
    {"messsages":[HumanMessage(content="My name is Ratnesh Bhardwaj!")],"language":"Hindi"}
)

response.content

'नमस्ते रत्नेश भारद्वाज जी! मैं आपकी मदद करने के लिए तैयार हूँ। मुझसे क्या पूछना चाहते हैं?'

## Managing the Conversation History

In [49]:
from langchain_core.messages import trim_messages, SystemMessage

trimmer = trim_messages(
    max_tokens=45,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human"
)

messsages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="Hi, I am Ratnesh"),
    AIMessage(content="Hi"),
    HumanMessage(content="I like chocolate icecream"),
    AIMessage(content="nice"),
    HumanMessage(content="What's 2+2?"),
    AIMessage(content="4"),
    HumanMessage(content="thanks!"),
    AIMessage(content="no problem"),
    HumanMessage(content="Having Fun?"),
    AIMessage(content="Yes!"),
]

In [51]:
trimmer.invoke(messsages)

[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like chocolate icecream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}, tool_calls=[], invalid_tool_calls=[]),
 HumanMessage(content="What's 2+2?", additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}, tool_calls=[], invalid_tool_calls=[]),
 HumanMessage(content='thanks!', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem', additional_kwargs={}, response_metadata={}, tool_calls=[], invalid_tool_calls=[]),
 HumanMessage(content='Having Fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Yes!', additional_kwargs={}, response_metadata={}, tool_calls=[], invalid_tool_calls=[])]

In [55]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough

chain = (
    RunnablePassthrough.assign(messages=itemgetter("messsages")|trimmer)
    | prompt
    | model
)

In [56]:
response = chain.invoke(
    {
    "messsages":messsages + [HumanMessage(content="what is my favorite Icecream?")],
    "language":"English"
    }
)

response.content

'Your favorite ice cream is Chocolate!'

In [59]:
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messsages"
)

config = {
    "configurable":{
        "session_id":"chat5"
    }
}

In [60]:
with_message_history.invoke(
    {
    "messsages":messsages + [HumanMessage(content="what is my favorite Icecream?")],
    "language":"English"
    },
    config=config
)

AIMessage(content='Your favorite ice cream is Chocolate!', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 149, 'total_tokens': 157, 'completion_time': 0.036226235, 'completion_tokens_details': None, 'prompt_time': 0.016501052, 'prompt_tokens_details': None, 'queue_time': 0.053682417, 'total_time': 0.052727287}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_dae98b5ecb', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019bb885-520f-7e50-99bd-23898a9fb15d-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 149, 'output_tokens': 8, 'total_tokens': 157})